
# **Deep-Learning Based Human Physical Activity Recognition with Wearable Sensor Data**

The following code implements gender recognition utilising wearable sensor data for multiple activities and sensor placements. This particular example utilises all the six measured signals (accelero x, y, z and gyro x, y, z) and applies end-to-end deep learning, fusing readings from multiple sensors for each activity via a multi-head 1D CNN.

In [ ]:
# Import Libraries
%matplotlib inline
import os
import io
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import itertools
import scipy.io
import glob
from zipfile import ZipFile
from datetime import datetime
from collections import Counter
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import LeaveOneGroupOut
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import GlobalAveragePooling1D
from keras.layers import Input, Add, Permute, Reshape, multiply
from keras.layers import Concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from keras.utils.vis_utils import plot_model
from matplotlib import pyplot, image
from scipy import stats
from keras.layers.merge import concatenate
print(tf.__version__)

2.8.0


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/HAR Research Project/Datasets/Research Dataset')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/HAR Research Project/Datasets


## Data Ingestion

In [ ]:
'''
The following section reads the accelerometer and gyroscope sensor readings of several different activities conducted by test subjects into a Pandas data frame.
The dataset consists of 135 subject-trial groups, with 45 subjects undergoing three trials each for seven different activities simultaneously measured by five sensors.
The dataset has been divided into seven activity folders, whereby each folder consists of .CSV files named as per the following convention:
'activityNumber_subjectName_trialNumber_startTimestamp_sensorNumber'
'''

archive_ = globals()
files_ = globals()
df_ = globals()

archive = []
all_dfs = []

# Loop through all the seven activity folders
for i in range(1,8):
  folder_path = f"{i}" + '.zip'
  # Read ZIP folder
  archive_[f"{i}"] = ZipFile(folder_path, 'r')
  archive.append(archive_[f"{i}"])
  # Read names of the files in ZIP folder
  files_[f"{i}"] = archive_[f"{i}"].namelist()
  dfs = []
  # Loop through all the files in the activity ZIP folder
  for file in files_[f"{i}"][1:]: # skip .DS_Store
    # Read file
    frame = pd.read_csv(archive[i-1].open(file), header=None)
    # Add filename as column
    frame['filename'] = os.path.basename(file)
    # Append all dataframes from within the activity
    dfs.append(frame)
  # Concatenate all dataframes from within the activity
  df_[f"{i}"] = pd.concat(dfs,ignore_index=True)
  # Append all dataframes from all activites
  all_dfs.append(df_[f"{i}"])
# Concatenate all dataframes from all activities
raw_df = pd.concat(all_dfs)

In [ ]:
# Read the subject characteristics into a Pandas data frame.
gender_df = pd.read_csv('subject_chars_sheet1.csv')

## Data Pre-Processing

In [ ]:
# Rename columns
full_df = raw_df.rename(columns={0: 'x_accelero', 1: 'y_accelero', 2: 'z_accelero', 3 : 'x_gyro', 4: 'y_gyro', 5: 'z_gyro'}, errors="raise")

# Split the filename into seperate columns
full_df[['activity','subject_name', 'trial_number', 'timestamp', 'sensor_position']] = full_df['filename'].str.split('_',expand=True)

# Drop irrelevant columns
full_df[['sensor_position','file_type']] = full_df['sensor_position'].str.split('.',expand=True)

# Merge the sensor readings' dataframe with the subject characteristcs' dataframe
full_df.drop(columns=['filename', 'file_type'], inplace=True)
gender_df = pd.read_csv('subject_chars_sheet1.csv')
full_gd_df = full_df.merge(gender_df, on='subject_name', how='left')

# Convert datatype for sensor readings
full_gd_df['x_accelero'] = pd.to_numeric(full_gd_df['x_accelero'])
full_gd_df['y_accelero'] = pd.to_numeric(full_gd_df['y_accelero'])
full_gd_df['z_accelero'] = pd.to_numeric(full_gd_df['z_accelero'])

full_gd_df['x_gyro'] = pd.to_numeric(full_gd_df['x_gyro'])

full_gd_df['y_gyro'] = full_gd_df['y_gyro'].astype('string')
full_gd_df['y_gyro'] = full_gd_df['y_gyro'].apply(lambda x: x[:-2] if x[-2:] == ".1" else x)
full_gd_df['y_gyro'] = pd.to_numeric(full_gd_df['y_gyro'])

full_gd_df['z_gyro'] = full_gd_df['z_gyro'].astype('string')
full_gd_df['z_gyro'] = full_gd_df['z_gyro'].apply(lambda x: x[:-2] if x[-2:] == ".1" else x)
full_gd_df['z_gyro'] = pd.to_numeric(full_gd_df['z_gyro'])

full_gd_df['sensor_position'] = pd.to_numeric(full_gd_df['sensor_position'])
full_gd_df['activity'] = pd.to_numeric(full_gd_df['activity'])
full_gd_df['Gender Code'] = pd.to_numeric(full_gd_df['Gender Code'])

full_gd_df["Subject_Trial_Number"] = full_gd_df["subject_name"] + full_gd_df["trial_number"]
full_gd_df["Subject_Trial_Sensor_Number"] = full_gd_df["subject_name"] + full_gd_df["trial_number"] + full_gd_df["sensor_position"].astype(str)

# Merge subject name and trial number
full_gd_df["Subject_Trial_Sensor_Activity_Number"] = full_gd_df["subject_name"] + full_gd_df["trial_number"] + full_gd_df["sensor_position"].astype(str) + full_gd_df["activity"].astype(str)

# This additional line drops subject records where the data is not given or is incomplete for all trials and sensors. 
# Unlike previous experiments where one subject_trial is left out for 1 sensor at a time, and rest are reshaped and computed, in this experiment sensors are used in parallel
# Leave one out returns the indices, therefore if data is missing for a parallel sensor the method would return an error
full_gd_df = full_gd_df[(full_gd_df.subject_name != 'Cat') & (full_gd_df.subject_name != 'cheryl') & (full_gd_df.subject_name != 'drdang') & (full_gd_df.subject_name != 'gdil') & (full_gd_df.subject_name != 'thanh') & (full_gd_df.subject_name != 'tuong')]
full_gd_df = full_gd_df.reset_index(drop=True)
full_gd_df


In [ ]:
# The following code ensures all sensors have the same timesteps, before LOGO is called
dfs = []
activity_list = set(full_gd_df.Subject_Trial_Sensor_Activity_Number)
for activity in activity_list:
  partial_df = full_gd_df[full_gd_df.Subject_Trial_Sensor_Activity_Number == activity]
  df_0 = pd.DataFrame([0]*(3000 - partial_df.shape[0]))
  df = partial_df.append(df_0, ignore_index=True)
  for column in ['Gender', 'Gender Code', 'Subject_Trial_Number', 'Subject_Trial_Sensor_Activity_Number', 'Subject_Trial_Sensor_Number', 
                 'activity', 'sensor_position', 'subject_name', 'trial_number']:
                 df[column].fillna(df[column].mode()[0], inplace=True)
  df = df.replace(np.nan,0)
  dfs.append(df)
final_df = pd.concat(dfs,ignore_index=True)
final_df = final_df.reset_index(drop=True)
final_df

## Function Definition

### Learning Rate Scheduler

In [ ]:
def scheduler(epoch, lr):
  ''''
  Returns the initial learning rate for the first ten epochs and then decreases it exponentially afterward.
  '''
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

# Create a callback for the Learning Rate Scheduler
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler) 

### Plot Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Accent)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    
    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure

## 2 Parallel Sensors 

### CNN Model

In [ ]:
# Multi-headed 1D CNN
def cnn_model_creation(n_timesteps, n_features):
    # head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs1) #change kernel size to 5 
    bn1 = BatchNormalization()(conv1)
    pool1 = GlobalAveragePooling1D()(bn1) 
    flat1 = Flatten()(pool1)

    # head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs2)
    bn2 = BatchNormalization()(conv2)
    pool2 = GlobalAveragePooling1D()(bn2)
    flat2 = Flatten()(pool2)

    # merge
    merged = concatenate([flat1, flat2])

    # interpretation
    dense1 = Dense(16, activation='relu', kernel_regularizer='l2')(merged) #16
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)

    # save a plot of the model
    # plot_model(model, show_shapes=True, to_file='multichannel.png')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

### Reshaping Signal Sequences into Frames with the Sliding Window Method

In [ ]:
def get_frames(X, Y):
  
  N_FEATURES = 6

  frames = []
  labels = []
  for Subject_Trial_Number_Encoded in set(X.Subject_Trial_Number_Encoded):  #for each group
    current_frame = X.loc[X.Subject_Trial_Number_Encoded == Subject_Trial_Number_Encoded]  #get the all the frames for that group
    start_index = min(current_frame.index)
    end_index = max(current_frame.index) + 1
    frame_size = len(current_frame)

    ax = X['x_accelero'].values[start_index: end_index] 
    ay = X['y_accelero'].values[start_index: end_index]
    az = X['z_accelero'].values[start_index: end_index]
    gx = X['x_gyro'].values[start_index: end_index] 
    gy = X['y_gyro'].values[start_index: end_index]
    gz = X['z_gyro'].values[start_index: end_index]

    # Retrieve the most often used label in this segment
    label = stats.mode(Y[start_index: end_index])[0][0]
    
    frames.append([ax, ay, az, gx, gy, gz])
    labels.append(label)
  # returns frames of samples, each sample(group) with three features, each feature with n timesteps 28*3*1729 i.e. groups/samples * features * timesteps

  # As the frame size differes for each group, the frames are padded
  padded_frames = []
  for row in frames:
    shape = np.shape(row)
    padded_array = np.zeros((6, 3000)) 
    padded_array[:shape[0],:shape[1]] = row
    padded_frames.append(padded_array)

  # return exactly the same shape as above but padded so each group has same number of steps - might not need this for this experiment! *CHECK*

  # Bring the segments into a better shape
  reshaped_padded_frames = np.transpose(padded_frames, (0, 2, 1))
  reshaped_labels = np.asarray(labels)
  
  # finally converts the  28*3*1729  -> 28*1729*3 i.e. groups/samples * timestep * features
  return reshaped_padded_frames, reshaped_labels

### Training Model

In [ ]:
def get_scores(sensor_position_number, activity_number, full_gd_df): 

  partial_df = final_df.drop(columns=[0, 
                                    'subject_name', 'trial_number','timestamp', 
                                    'Age', 'Age Group', 'Gender', 'Weight', 'Height', 'BMI',
                                    'Subject_Trial_Sensor_Activity_Number', 'Subject_Trial_Sensor_Number']) #DROP MORE COLUMNS
  le = preprocessing.LabelEncoder()
  partial_df['Subject_Trial_Number_Encoded'] = le.fit_transform(partial_df['Subject_Trial_Number'])
  partial_df = partial_df.drop(columns=['Subject_Trial_Number'])
  partial_df_original = partial_df[partial_df["activity"] == activity_number]

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 1
  partial_df = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[0]]
  partial_df = partial_df.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train = partial_df[partial_df['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test = partial_df[partial_df['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN = partial_df_train[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN = partial_df_train['Gender Code']
  X_TRAIN = X_TRAIN.to_numpy() # for LOGO
  y_TRAIN = y_TRAIN.to_numpy() # for LOGO

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 2
  partial_df_2 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[1]]
  partial_df_2 = partial_df_2.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_2 = partial_df_train_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_2 = partial_df_train_2['Gender Code']
  X_TRAIN_2 = X_TRAIN_2.to_numpy() # for LOGO
  y_TRAIN_2 = y_TRAIN_2.to_numpy() # for LOGO

  # Both sensor 1 and 2 train sets should have same indices that can be referenced by logo
  #-------------------------------------------------------------------------------------------

  # Define TEST X and y variables
  X_TEST = partial_df_test[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST = partial_df_test['Gender Code'].reset_index(drop=True)

  reshaped_X_test, reshaped_y_test = get_frames(X_TEST, y_TEST) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Define TEST X and y variables
  X_TEST_2 = partial_df_test_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_2 = partial_df_test_2['Gender Code'].reset_index(drop=True)

  reshaped_X_test_2, reshaped_y_test_2 = get_frames(X_TEST_2, y_TEST_2) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  groups = partial_df_train['Subject_Trial_Number_Encoded']
  logo = LeaveOneGroupOut()
  split_number = logo.get_n_splits(X_TRAIN, y_TRAIN, groups)
  groups = groups.to_numpy()

  #-------------------------------------------------------------------------------------------

  acc_per_fold = []
  loss_per_fold = []
  # data for the confusion matrix
  cm_holder_per_fold = []
  # create empty lists for later
  y_true, y_pred = list(), list()

  for train_ix, val_ix in logo.split(X_TRAIN, y_TRAIN, groups):
      # split data
      X_train, X_val = X_TRAIN[train_ix, :], X_TRAIN[val_ix, :]
      y_train, y_val = y_TRAIN[train_ix], y_TRAIN[val_ix] 

      X_train = pd.DataFrame(data = X_train, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val = pd.DataFrame(data = X_val, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train = pd.DataFrame(data = y_train)
      y_val = pd.DataFrame(data = y_val) 

      reshaped_X_train, reshaped_y_train = get_frames(X_train, y_train) 
      reshaped_X_val, reshaped_y_val = get_frames(X_val, y_val) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_2, X_val_2 = X_TRAIN_2[train_ix, :], X_TRAIN_2[val_ix, :]
      y_train_2, y_val_2 = y_TRAIN_2[train_ix], y_TRAIN_2[val_ix] 

      X_train_2 = pd.DataFrame(data = X_train_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_2 = pd.DataFrame(data = X_val_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_2 = pd.DataFrame(data = y_train_2)
      y_val_2 = pd.DataFrame(data = y_val_2) 

      reshaped_X_train_2, reshaped_y_train_2 = get_frames(X_train_2, y_train_2) 
      reshaped_X_val_2, reshaped_y_val_2 = get_frames(X_val_2, y_val_2) 

      #-------------------------------------------------------------------------------------------
      
      # defining some input variables
      n_timesteps, n_features, n_outputs = reshaped_X_train.shape[1], reshaped_X_val.shape[2], reshaped_y_train.shape[1]

      # getting the model
      model = cnn_model_creation(n_timesteps, n_features)

      #model.summary()
      #plot_model(model, to_file='model_plot_3.png', show_shapes=True, show_layer_names=True) 

      # fit model
      history = model.fit([reshaped_X_train,reshaped_X_train_2], reshaped_y_train,
                epochs=50,
                verbose=0,
                callbacks=[lr_callback],
                validation_data=([reshaped_X_val,reshaped_X_val_2], reshaped_y_val))
      
      test_scores = model.evaluate([reshaped_X_test, reshaped_X_test_2],reshaped_y_test, verbose=0)
      acc_per_fold.append(test_scores[1] * 100)
      loss_per_fold.append(test_scores[0])

      # Use the model to predict the values from the test data.
      predictions_ = model.predict([reshaped_X_test, reshaped_X_test_2])
      # Take the class with the highest probability from the test predictions
      predictions = np.where(predictions_ > 0.5, 1, 0)
      y_pred.append(predictions)
      # Calculate the confusion matrix using sklearn.metrics
      cm = metrics.confusion_matrix(reshaped_y_test, predictions)
      
      # append the confusion matrix of this fold
      cm_holder_per_fold.append(cm)

      # store ground truth and predicted values
      y_true.append(reshaped_y_test) #[0])
      y_pred.append(predictions) #[0])

  class_names = ['0', '1']

  # confusion matrix per fold
  sum_cm_holder_per_fold = []
  cm_shape = np.array([len(class_names),len(class_names)])
  for k in range(len(cm_holder_per_fold)):
      cm_mask = np.zeros(cm_shape)
      cm_mask[:cm_holder_per_fold[k].shape[0], :cm_holder_per_fold[k].shape[1]] = cm_holder_per_fold[k]
      sum_cm_holder_per_fold.append(cm_mask)

  sum_cm_per_fold = sum(sum_cm_holder_per_fold)
  figure = plot_confusion_matrix(sum_cm_per_fold, class_names=class_names)
  plt.show()  
  
  mean_accuracy = np.mean(acc_per_fold)
  mean_std = np.std(acc_per_fold)
  mean_loss = np.mean(loss_per_fold)
  print('Sensor:', sensor_position_number, 'Activity:', activity_number)
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  return mean_accuracy

### Call the Train Function

In [ ]:
test_scores = []
activity_list = [1, 2, 3, 4, 5, 6, 7]
sensor_list = [
               [1,2]
              ,[1,3]
              ,[1,4]
              ,[1,5]
              ,[2,3]
              ,[2,4]
              ,[2,5]
              ,[3,4]
              ,[3,5]
              ,[4,5]
               ]
for activity in activity_list:
  for sensor in sensor_list:
    mean_accuracy = get_scores(sensor, activity, final_df)     
    test_scores.append((activity, sensor, mean_accuracy))

test_scores

### Plot the ROC-AUC Graphs

In [ ]:
from sklearn.metrics import roc_curve
fpr1, tpr1, thresh1 = roc_curve(reshaped_y_test, predictions_, pos_label=1)

# roc curve for tpr = fpr 
random_probs = [0 for i in range(len(reshaped_y_test))]
p_fpr, p_tpr, _ = roc_curve(reshaped_y_test, random_probs, pos_label=1)

# matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# plot roc curves
plt.plot(fpr1, tpr1, linestyle='--',color='orange', label='1D CNN')
plt.plot(p_fpr, p_tpr, linestyle='--', color='blue')
# title
plt.title('ROC curve')
# x label
plt.xlabel('False Positive Rate')
# y label
plt.ylabel('True Positive rate')

plt.legend(loc='best')
plt.savefig('ROC',dpi=300)
plt.show();

## 3 Parallel Sensors

### CNN Model

In [ ]:
# Multi-headed 1D CNN
def cnn_model_creation(n_timesteps, n_features):
    # head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs1)
    bn1 = BatchNormalization()(conv1)
    pool1 = GlobalAveragePooling1D()(bn1) 
    flat1 = Flatten()(pool1)

    # head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs2)
    bn2 = BatchNormalization()(conv2)
    pool2 = GlobalAveragePooling1D()(bn2)
    flat2 = Flatten()(pool2)

    # head 3
    inputs3 = Input(shape=(n_timesteps,n_features))
    conv3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs3)
    bn3 = BatchNormalization()(conv3)
    pool3 = GlobalAveragePooling1D()(bn3)
    flat3 = Flatten()(pool3)

    # merge
    merged = concatenate([flat1,flat2,flat3])

    # interpretation
    dense1 = Dense(16, activation='relu')(merged) #16
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

    # save a plot of the model
    # plot_model(model, show_shapes=True, to_file='multichannel.png')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

### Reshaping Signal Sequences into Frames with the Sliding Window Method

In [ ]:
def get_frames(X, Y):
  
  N_FEATURES = 6

  frames = []
  labels = []
  for Subject_Trial_Number_Encoded in set(X.Subject_Trial_Number_Encoded):  #for each group
    current_frame = X.loc[X.Subject_Trial_Number_Encoded == Subject_Trial_Number_Encoded]  #get the all the frames for that group
    start_index = min(current_frame.index)
    end_index = max(current_frame.index) + 1
    frame_size = len(current_frame)

    ax = X['x_accelero'].values[start_index: end_index] 
    ay = X['y_accelero'].values[start_index: end_index]
    az = X['z_accelero'].values[start_index: end_index]
    gx = X['x_gyro'].values[start_index: end_index] 
    gy = X['y_gyro'].values[start_index: end_index]
    gz = X['z_gyro'].values[start_index: end_index]

    # Retrieve the most often used label in this segment
    label = stats.mode(Y[start_index: end_index])[0][0]
    
    frames.append([ax, ay, az, gx, gy, gz])
    labels.append(label)
  # returns frames of samples, each sample(group) with three features, each feature with n timesteps 28*3*1729 i.e. groups/samples * features * timesteps

  # As the frame size differes for each group, the frames are padded
  padded_frames = []
  for row in frames:
    shape = np.shape(row)
    padded_array = np.zeros((6, 3000)) 
    padded_array[:shape[0],:shape[1]] = row
    padded_frames.append(padded_array)

  # return exactly the same shape as above but padded so each group has same number of steps - might not need this for this experiment! *CHECK*

  # Bring the segments into a better shape
  reshaped_padded_frames = np.transpose(padded_frames, (0, 2, 1))
  reshaped_labels = np.asarray(labels)
  
  # finally converts the  28*3*1729  -> 28*1729*3 i.e. groups/samples * timestep * features
  return reshaped_padded_frames, reshaped_labels

### Training Model

In [ ]:
def get_scores(sensor_position_number, activity_number, full_gd_df): 

  partial_df = final_df.drop(columns=[0, 
                                    'subject_name', 'trial_number','timestamp', 
                                    'Age', 'Age Group', 'Gender', 'Weight', 'Height', 'BMI',
                                    'Subject_Trial_Sensor_Activity_Number', 'Subject_Trial_Sensor_Number']) #DROP MORE COLUMNS
  le = preprocessing.LabelEncoder()
  partial_df['Subject_Trial_Number_Encoded'] = le.fit_transform(partial_df['Subject_Trial_Number'])
  partial_df = partial_df.drop(columns=['Subject_Trial_Number'])
  partial_df_original = partial_df[partial_df["activity"] == activity_number]

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 1
  partial_df = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[0]]
  partial_df = partial_df.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train = partial_df[partial_df['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test = partial_df[partial_df['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN = partial_df_train[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN = partial_df_train['Gender Code']
  X_TRAIN = X_TRAIN.to_numpy() # for LOGO
  y_TRAIN = y_TRAIN.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST = partial_df_test[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST = partial_df_test['Gender Code'].reset_index(drop=True)

  reshaped_X_test, reshaped_y_test = get_frames(X_TEST, y_TEST) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 2
  partial_df_2 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[1]]
  partial_df_2 = partial_df_2.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_2 = partial_df_train_2[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_2 = partial_df_train_2['Gender Code']
  X_TRAIN_2 = X_TRAIN_2.to_numpy() # for LOGO
  y_TRAIN_2 = y_TRAIN_2.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_2 = partial_df_test_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_2 = partial_df_test_2['Gender Code'].reset_index(drop=True)

  reshaped_X_test_2, reshaped_y_test_2 = get_frames(X_TEST_2, y_TEST_2) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 3
  partial_df_3 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[2]]
  partial_df_3 = partial_df_3.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_3[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_3 = partial_df_train_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_3 = partial_df_train_3['Gender Code']
  X_TRAIN_3 = X_TRAIN_3.to_numpy() # for LOGO
  y_TRAIN_3 = y_TRAIN_3.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_3 = partial_df_test_3[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_3 = partial_df_test_3['Gender Code'].reset_index(drop=True)

  reshaped_X_test_3, reshaped_y_test_3 = get_frames(X_TEST_3, y_TEST_3) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  groups = partial_df_train['Subject_Trial_Number_Encoded']
  logo = LeaveOneGroupOut()
  split_number = logo.get_n_splits(X_TRAIN, y_TRAIN, groups)
  groups = groups.to_numpy()

  #-------------------------------------------------------------------------------------------

  acc_per_fold = []
  loss_per_fold = []

  for train_ix, val_ix in logo.split(X_TRAIN, y_TRAIN, groups):
      # split data
      X_train, X_val = X_TRAIN[train_ix, :], X_TRAIN[val_ix, :]
      y_train, y_val = y_TRAIN[train_ix], y_TRAIN[val_ix] 

      X_train = pd.DataFrame(data = X_train, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val = pd.DataFrame(data = X_val, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train = pd.DataFrame(data = y_train)
      y_val = pd.DataFrame(data = y_val) 

      reshaped_X_train, reshaped_y_train = get_frames(X_train, y_train) 
      reshaped_X_val, reshaped_y_val = get_frames(X_val, y_val) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_2, X_val_2 = X_TRAIN_2[train_ix, :], X_TRAIN_2[val_ix, :]
      y_train_2, y_val_2 = y_TRAIN_2[train_ix], y_TRAIN_2[val_ix] 

      X_train_2 = pd.DataFrame(data = X_train_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_2 = pd.DataFrame(data = X_val_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_2 = pd.DataFrame(data = y_train_2)
      y_val_2 = pd.DataFrame(data = y_val_2) 

      reshaped_X_train_2, reshaped_y_train_2 = get_frames(X_train_2, y_train_2) 
      reshaped_X_val_2, reshaped_y_val_2 = get_frames(X_val_2, y_val_2) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_3, X_val_3 = X_TRAIN_3[train_ix, :], X_TRAIN_3[val_ix, :]
      y_train_3, y_val_3 = y_TRAIN_3[train_ix], y_TRAIN_3[val_ix] 

      X_train_3 = pd.DataFrame(data = X_train_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_3 = pd.DataFrame(data = X_val_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_3 = pd.DataFrame(data = y_train_3)
      y_val_3 = pd.DataFrame(data = y_val_3) 

      reshaped_X_train_3, reshaped_y_train_3 = get_frames(X_train_3, y_train_3) 
      reshaped_X_val_3, reshaped_y_val_3 = get_frames(X_val_3, y_val_3) 

      #-------------------------------------------------------------------------------------------

      # defining some input variables
      n_timesteps, n_features, n_outputs = reshaped_X_train.shape[1], reshaped_X_val.shape[2], reshaped_y_train.shape[1]

      # getting the model
      model = cnn_model_creation(n_timesteps, n_features)

      # fit model
      history = model.fit([reshaped_X_train,reshaped_X_train_2,reshaped_X_train_3], reshaped_y_train,
                epochs=50,
                verbose=0,
                callbacks=[lr_callback],
                validation_data=([reshaped_X_val,reshaped_X_val_2,reshaped_X_val_3], reshaped_y_val)) # Chnage number of epochs!
      
      test_scores = model.evaluate([reshaped_X_test, reshaped_X_test_2,reshaped_X_test_3],reshaped_y_test, verbose=0)
      acc_per_fold.append(test_scores[1] * 100)
      loss_per_fold.append(test_scores[0])
  mean_accuracy = np.mean(acc_per_fold)
  mean_std = np.std(acc_per_fold)
  mean_loss = np.mean(loss_per_fold)
  print('Sensor:', sensor_position_number, 'Activity:', activity_number)
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  return mean_accuracy

### Call the Train Function

In [ ]:
test_scores = []
activity_list = [1, 2, 3, 4, 5, 6, 7]
sensor_list = [
               [1,2,3]
              ,[1,2,4]
              ,[1,2,5]
              ,[1,3,4]
              ,[1,3,5]
              ,[1,4,5]
              ,[2,3,4]
              ,[2,3,5]
              ,[2,4,5]
              ,[3,4,5]
               ]
for activity in activity_list:
  for sensor in sensor_list:
    mean_accuracy = get_scores(sensor, activity, final_df)     
    test_scores.append((activity, sensor, mean_accuracy))

test_scores

## 4 Parallel Sensors

### CNN Model

In [ ]:
# Multi-headed 1D CNN
def cnn_model_creation(n_timesteps, n_features):
    # head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs1)
    bn1 = BatchNormalization()(conv1)
    pool1 = GlobalAveragePooling1D()(bn1) 
    flat1 = Flatten()(pool1)

    # head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs2)
    bn2 = BatchNormalization()(conv2)
    pool2 = GlobalAveragePooling1D()(bn2)
    flat2 = Flatten()(pool2)

    # head 3
    inputs3 = Input(shape=(n_timesteps,n_features))
    conv3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs3)
    bn3 = BatchNormalization()(conv3)
    pool3 = GlobalAveragePooling1D()(bn3)
    flat3 = Flatten()(pool3)

    # head 4
    inputs4 = Input(shape=(n_timesteps,n_features))
    conv4 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs4)
    bn4 = BatchNormalization()(conv4)
    pool4 = GlobalAveragePooling1D()(bn4)
    flat4 = Flatten()(pool4)

    # merge
    merged = concatenate([flat1,flat2,flat3,flat4])

    # interpretation
    dense1 = Dense(16, activation='relu')(merged) #16
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4], outputs=outputs)

    # save a plot of the model
    # plot_model(model, show_shapes=True, to_file='multichannel.png')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

### Reshaping Signal Sequences into Frames with the Sliding Window Method

In [ ]:
def get_frames(X, Y):
  
  N_FEATURES = 6

  frames = []
  labels = []
  for Subject_Trial_Number_Encoded in set(X.Subject_Trial_Number_Encoded):  #for each group
    current_frame = X.loc[X.Subject_Trial_Number_Encoded == Subject_Trial_Number_Encoded]  #get the all the frames for that group
    start_index = min(current_frame.index)
    end_index = max(current_frame.index) + 1
    frame_size = len(current_frame)

    ax = X['x_accelero'].values[start_index: end_index] 
    ay = X['y_accelero'].values[start_index: end_index]
    az = X['z_accelero'].values[start_index: end_index]
    gx = X['x_gyro'].values[start_index: end_index] 
    gy = X['y_gyro'].values[start_index: end_index]
    gz = X['z_gyro'].values[start_index: end_index]

    # Retrieve the most often used label in this segment
    label = stats.mode(Y[start_index: end_index])[0][0]
    
    frames.append([ax, ay, az, gx, gy, gz])
    labels.append(label)
  # returns frames of samples, each sample(group) with three features, each feature with n timesteps 28*3*1729 i.e. groups/samples * features * timesteps

  # As the frame size differes for each group, the frames are padded
  padded_frames = []
  for row in frames:
    shape = np.shape(row)
    padded_array = np.zeros((6, 3000)) 
    padded_array[:shape[0],:shape[1]] = row
    padded_frames.append(padded_array)

  # return exactly the same shape as above but padded so each group has same number of steps - might not need this for this experiment! *CHECK*

  # Bring the segments into a better shape
  reshaped_padded_frames = np.transpose(padded_frames, (0, 2, 1))
  reshaped_labels = np.asarray(labels)
  
  # finally converts the  28*3*1729  -> 28*1729*3 i.e. groups/samples * timestep * features
  return reshaped_padded_frames, reshaped_labels

### Training Model

In [ ]:
def get_scores(sensor_position_number, activity_number, full_gd_df): 

  partial_df = final_df.drop(columns=[0, 
                                    'subject_name', 'trial_number','timestamp', 
                                    'Age', 'Age Group', 'Gender', 'Weight', 'Height', 'BMI',
                                    'Subject_Trial_Sensor_Activity_Number', 'Subject_Trial_Sensor_Number']) #DROP MORE COLUMNS
  le = preprocessing.LabelEncoder()
  partial_df['Subject_Trial_Number_Encoded'] = le.fit_transform(partial_df['Subject_Trial_Number'])
  partial_df = partial_df.drop(columns=['Subject_Trial_Number'])
  partial_df_original = partial_df[partial_df["activity"] == activity_number]

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 1
  partial_df = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[0]]
  partial_df = partial_df.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train = partial_df[partial_df['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test = partial_df[partial_df['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN = partial_df_train[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN = partial_df_train['Gender Code']
  X_TRAIN = X_TRAIN.to_numpy() # for LOGO
  y_TRAIN = y_TRAIN.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST = partial_df_test[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST = partial_df_test['Gender Code'].reset_index(drop=True)

  reshaped_X_test, reshaped_y_test = get_frames(X_TEST, y_TEST) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 2
  partial_df_2 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[1]]
  partial_df_2 = partial_df_2.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_2 = partial_df_train_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_2 = partial_df_train_2['Gender Code']
  X_TRAIN_2 = X_TRAIN_2.to_numpy() # for LOGO
  y_TRAIN_2 = y_TRAIN_2.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_2 = partial_df_test_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_2 = partial_df_test_2['Gender Code'].reset_index(drop=True)

  reshaped_X_test_2, reshaped_y_test_2 = get_frames(X_TEST_2, y_TEST_2) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 3
  partial_df_3 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[2]]
  partial_df_3 = partial_df_3.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_3 = partial_df_train_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_3 = partial_df_train_3['Gender Code']
  X_TRAIN_3 = X_TRAIN_3.to_numpy() # for LOGO
  y_TRAIN_3 = y_TRAIN_3.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_3 = partial_df_test_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_3 = partial_df_test_3['Gender Code'].reset_index(drop=True)

  reshaped_X_test_3, reshaped_y_test_3 = get_frames(X_TEST_3, y_TEST_3) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  # Get data for sensor 4
  partial_df_4 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[3]]
  partial_df_4 = partial_df_4.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_4 = partial_df_4[partial_df_4['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_4 = partial_df_4[partial_df_4['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_4 = partial_df_train_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_4 = partial_df_train_4['Gender Code']
  X_TRAIN_4 = X_TRAIN_4.to_numpy() # for LOGO
  y_TRAIN_4 = y_TRAIN_4.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_4 = partial_df_test_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_4 = partial_df_test_4['Gender Code'].reset_index(drop=True)

  reshaped_X_test_4, reshaped_y_test_4 = get_frames(X_TEST_4, y_TEST_4) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  groups = partial_df_train['Subject_Trial_Number_Encoded']
  logo = LeaveOneGroupOut()
  split_number = logo.get_n_splits(X_TRAIN, y_TRAIN, groups)
  groups = groups.to_numpy()

  #-------------------------------------------------------------------------------------------

  acc_per_fold = []
  loss_per_fold = []

  for train_ix, val_ix in logo.split(X_TRAIN, y_TRAIN, groups):
      # split data
      X_train, X_val = X_TRAIN[train_ix, :], X_TRAIN[val_ix, :]
      y_train, y_val = y_TRAIN[train_ix], y_TRAIN[val_ix] 

      X_train = pd.DataFrame(data = X_train, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val = pd.DataFrame(data = X_val, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train = pd.DataFrame(data = y_train)
      y_val = pd.DataFrame(data = y_val) 

      reshaped_X_train, reshaped_y_train = get_frames(X_train, y_train) 
      reshaped_X_val, reshaped_y_val = get_frames(X_val, y_val) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_2, X_val_2 = X_TRAIN_2[train_ix, :], X_TRAIN_2[val_ix, :]
      y_train_2, y_val_2 = y_TRAIN_2[train_ix], y_TRAIN_2[val_ix] 

      X_train_2 = pd.DataFrame(data = X_train_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro',  'Subject_Trial_Number_Encoded'])
      X_val_2 = pd.DataFrame(data = X_val_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_2 = pd.DataFrame(data = y_train_2)
      y_val_2 = pd.DataFrame(data = y_val_2) 

      reshaped_X_train_2, reshaped_y_train_2 = get_frames(X_train_2, y_train_2) 
      reshaped_X_val_2, reshaped_y_val_2 = get_frames(X_val_2, y_val_2) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_3, X_val_3 = X_TRAIN_3[train_ix, :], X_TRAIN_3[val_ix, :]
      y_train_3, y_val_3 = y_TRAIN_3[train_ix], y_TRAIN_3[val_ix] 

      X_train_3 = pd.DataFrame(data = X_train_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_3 = pd.DataFrame(data = X_val_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_3 = pd.DataFrame(data = y_train_3)
      y_val_3 = pd.DataFrame(data = y_val_3) 

      reshaped_X_train_3, reshaped_y_train_3 = get_frames(X_train_3, y_train_3) 
      reshaped_X_val_3, reshaped_y_val_3 = get_frames(X_val_3, y_val_3) 

      #-------------------------------------------------------------------------------------------

       # split data
      X_train_4, X_val_4 = X_TRAIN_4[train_ix, :], X_TRAIN_4[val_ix, :]
      y_train_4, y_val_4 = y_TRAIN_4[train_ix], y_TRAIN_4[val_ix] 

      X_train_4 = pd.DataFrame(data = X_train_4, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_4 = pd.DataFrame(data = X_val_4, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      y_train_4 = pd.DataFrame(data = y_train_4)
      y_val_4 = pd.DataFrame(data = y_val_4) 

      reshaped_X_train_4, reshaped_y_train_4 = get_frames(X_train_4, y_train_4) 
      reshaped_X_val_4, reshaped_y_val_4 = get_frames(X_val_4, y_val_4) 

      #-------------------------------------------------------------------------------------------

      # defining some input variables
      n_timesteps, n_features, n_outputs = reshaped_X_train.shape[1], reshaped_X_val.shape[2], reshaped_y_train.shape[1]

      # getting the model
      model = cnn_model_creation(n_timesteps, n_features)

      # fit model
      history = model.fit([reshaped_X_train,reshaped_X_train_2,reshaped_X_train_3,reshaped_X_train_4], reshaped_y_train,
                epochs=50,
                verbose=0,
                callbacks=[lr_callback],
                validation_data=([reshaped_X_val,reshaped_X_val_2,reshaped_X_val_3,reshaped_X_val_4], reshaped_y_val)) # Chnage number of epochs!
      
      test_scores = model.evaluate([reshaped_X_test, reshaped_X_test_2,reshaped_X_test_3,reshaped_X_test_4],reshaped_y_test, verbose=0)
      acc_per_fold.append(test_scores[1] * 100)
      loss_per_fold.append(test_scores[0])
  mean_accuracy = np.mean(acc_per_fold)
  mean_std = np.std(acc_per_fold)
  mean_loss = np.mean(loss_per_fold)
  print('Sensor:', sensor_position_number, 'Activity:', activity_number)
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  return mean_accuracy

### Call the Train Function

In [ ]:
test_scores = []
activity_list = [1, 2, 3, 4, 5, 6, 7]
sensor_list = [
               [1,2,3,4]
               ,[1,3,4,5]
               ,[1,4,5,2]
               ,[1,5,2,3]
               ,[2,3,4,5]
               ]
for activity in activity_list:
  for sensor in sensor_list:
    mean_accuracy = get_scores(sensor, activity, final_df)     
    test_scores.append((activity, sensor, mean_accuracy))

test_scores

## All 5 sensors in Parallel

### CNN Model

In [ ]:
# Multi-headed 1D CNN
def cnn_model_creation(n_timesteps, n_features):
    # head 1
    inputs1 = Input(shape=(n_timesteps,n_features))
    conv1 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs1)
    bn1 = BatchNormalization()(conv1)
    pool1 = GlobalAveragePooling1D()(bn1) 
    flat1 = Flatten()(pool1)

    # head 2
    inputs2 = Input(shape=(n_timesteps,n_features))
    conv2 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs2)
    bn2 = BatchNormalization()(conv2)
    pool2 = GlobalAveragePooling1D()(bn2)
    flat2 = Flatten()(pool2)

    # head 3
    inputs3 = Input(shape=(n_timesteps,n_features))
    conv3 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs3)
    bn3 = BatchNormalization()(conv3)
    pool3 = GlobalAveragePooling1D()(bn3)
    flat3 = Flatten()(pool3)

    # head 4
    inputs4 = Input(shape=(n_timesteps,n_features))
    conv4 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs4)
    bn4 = BatchNormalization()(conv4)
    pool4 = GlobalAveragePooling1D()(bn4)
    flat4 = Flatten()(pool4)

    # head 5
    inputs5 = Input(shape=(n_timesteps,n_features))
    conv5 = Conv1D(filters=64, kernel_size=5, activation='relu', padding='same')(inputs5)
    bn5 = BatchNormalization()(conv5)
    pool5 = GlobalAveragePooling1D()(bn5)
    flat5 = Flatten()(pool5)

    # merge
    merged = concatenate([flat1,flat2,flat3,flat4,flat5])

    # interpretation
    dense1 = Dense(16, activation='relu')(merged) #16
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4,inputs5], outputs=outputs)

    # save a plot of the model
    # plot_model(model, show_shapes=True, to_file='multichannel.png')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

### Reshaping Signal Sequences into Frames with the Sliding Window Method

In [ ]:
def get_frames(X, Y):
  
  N_FEATURES = 6

  frames = []
  labels = []
  for Subject_Trial_Number_Encoded in set(X.Subject_Trial_Number_Encoded):  #for each group
    current_frame = X.loc[X.Subject_Trial_Number_Encoded == Subject_Trial_Number_Encoded]  #get the all the frames for that group
    start_index = min(current_frame.index)
    end_index = max(current_frame.index) + 1
    frame_size = len(current_frame)

    ax = X['x_accelero'].values[start_index: end_index] 
    ay = X['y_accelero'].values[start_index: end_index]
    az = X['z_accelero'].values[start_index: end_index]
    gx = X['x_gyro'].values[start_index: end_index] 
    gy = X['y_gyro'].values[start_index: end_index]
    gz = X['z_gyro'].values[start_index: end_index]

    # Retrieve the most often used label in this segment
    label = stats.mode(Y[start_index: end_index])[0][0]
    
    frames.append([ax, ay, az, gx, gy, gz])
    labels.append(label)
  # returns frames of samples, each sample(group) with three features, each feature with n timesteps 28*3*1729 i.e. groups/samples * features * timesteps

  # As the frame size differes for each group, the frames are padded
  padded_frames = []
  for row in frames:
    shape = np.shape(row)
    padded_array = np.zeros((6, 3000)) 
    padded_array[:shape[0],:shape[1]] = row
    padded_frames.append(padded_array)

  # return exactly the same shape as above but padded so each group has same number of steps - might not need this for this experiment! *CHECK*

  # Bring the segments into a better shape
  reshaped_padded_frames = np.transpose(padded_frames, (0, 2, 1))
  reshaped_labels = np.asarray(labels)
  
  # finally converts the  28*3*1729  -> 28*1729*3 i.e. groups/samples * timestep * features
  return reshaped_padded_frames, reshaped_labels

### Training Model

In [ ]:
def get_scores(sensor_position_number, activity_number, full_gd_df): 

  partial_df = final_df.drop(columns=[0,  
                                    'subject_name', 'trial_number','timestamp', 
                                    'Age', 'Age Group', 'Gender', 'Weight', 'Height', 'BMI',
                                    'Subject_Trial_Sensor_Activity_Number', 'Subject_Trial_Sensor_Number']) #DROP MORE COLUMNS
  le = preprocessing.LabelEncoder()
  partial_df['Subject_Trial_Number_Encoded'] = le.fit_transform(partial_df['Subject_Trial_Number'])
  partial_df = partial_df.drop(columns=['Subject_Trial_Number'])
  partial_df_original = partial_df[partial_df["activity"] == activity_number]

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 1
  partial_df = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[0]]
  partial_df = partial_df.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train = partial_df[partial_df['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test = partial_df[partial_df['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN = partial_df_train[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']]
  y_TRAIN = partial_df_train['Gender Code']
  X_TRAIN = X_TRAIN.to_numpy() # for LOGO
  y_TRAIN = y_TRAIN.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST = partial_df_test[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST = partial_df_test['Gender Code'].reset_index(drop=True)

  reshaped_X_test, reshaped_y_test = get_frames(X_TEST, y_TEST) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 2
  partial_df_2 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[1]]
  partial_df_2 = partial_df_2.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_2 = partial_df_2[partial_df_2['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_2 = partial_df_train_2[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']]
  y_TRAIN_2 = partial_df_train_2['Gender Code']
  X_TRAIN_2 = X_TRAIN_2.to_numpy() # for LOGO
  y_TRAIN_2 = y_TRAIN_2.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_2 = partial_df_test_2[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_2 = partial_df_test_2['Gender Code'].reset_index(drop=True)

  reshaped_X_test_2, reshaped_y_test_2 = get_frames(X_TEST_2, y_TEST_2) # convert test data to frames

  #-------------------------------------------------------------------------------------------

  # Get data for sensor 3
  partial_df_3 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[2]]
  partial_df_3 = partial_df_3.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_3 = partial_df_3[partial_df_3['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_3 = partial_df_train_3[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded']]
  y_TRAIN_3 = partial_df_train_3['Gender Code']
  X_TRAIN_3 = X_TRAIN_3.to_numpy() # for LOGO
  y_TRAIN_3 = y_TRAIN_3.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_3 = partial_df_test_3[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_3 = partial_df_test_3['Gender Code'].reset_index(drop=True)

  reshaped_X_test_3, reshaped_y_test_3 = get_frames(X_TEST_3, y_TEST_3) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  # Get data for sensor 4
  partial_df_4 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[3]]
  partial_df_4 = partial_df_4.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_4[['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_4 = partial_df_4[partial_df_4['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_4 = partial_df_4[partial_df_4['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_4 = partial_df_train_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']]
  y_TRAIN_4 = partial_df_train_4['Gender Code']
  X_TRAIN_4 = X_TRAIN_4.to_numpy() # for LOGO
  y_TRAIN_4 = y_TRAIN_4.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_4 = partial_df_test_4[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_4 = partial_df_test_4['Gender Code'].reset_index(drop=True)

  reshaped_X_test_4, reshaped_y_test_4 = get_frames(X_TEST_4, y_TEST_4) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  # Get data for sensor 5
  partial_df_5 = partial_df_original[partial_df_original["sensor_position"] == sensor_position_number[4]]
  partial_df_5 = partial_df_5.drop(columns=['sensor_position', 'activity']).reset_index(drop=True) #this resets index each time for different sensor and activity combinations

  # Standardise
  scaler = StandardScaler()
  partial_df_5[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']] = scaler.fit_transform(partial_df_5[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro']])

  # Partition Train and Test Data
  partial_df_train_5 = partial_df_5[partial_df_5['Subject_Trial_Number_Encoded'] < 79].reset_index(drop=True) # check value!
  partial_df_test_5 = partial_df_5[partial_df_5['Subject_Trial_Number_Encoded'] >= 79].reset_index(drop=True) 

  # Define TRAIN X and y variables
  X_TRAIN_5 = partial_df_train_5[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']]
  y_TRAIN_5 = partial_df_train_5['Gender Code']
  X_TRAIN_5 = X_TRAIN_5.to_numpy() # for LOGO
  y_TRAIN_5 = y_TRAIN_5.to_numpy() # for LOGO

  # Define TEST X and y variables
  X_TEST_5 = partial_df_test_5[['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded']].reset_index(drop=True)
  y_TEST_5 = partial_df_test_5['Gender Code'].reset_index(drop=True)

  reshaped_X_test_5, reshaped_y_test_5 = get_frames(X_TEST_5, y_TEST_5) # convert test data to frames
  
  #-------------------------------------------------------------------------------------------

  groups = partial_df_train['Subject_Trial_Number_Encoded']
  logo = LeaveOneGroupOut()
  split_number = logo.get_n_splits(X_TRAIN, y_TRAIN, groups)
  groups = groups.to_numpy()

  #-------------------------------------------------------------------------------------------

  acc_per_fold = []
  loss_per_fold = []

  for train_ix, val_ix in logo.split(X_TRAIN, y_TRAIN, groups):
      # split data
      X_train, X_val = X_TRAIN[train_ix, :], X_TRAIN[val_ix, :]
      y_train, y_val = y_TRAIN[train_ix], y_TRAIN[val_ix] 

      X_train = pd.DataFrame(data = X_train, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      X_val = pd.DataFrame(data = X_val, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      y_train = pd.DataFrame(data = y_train)
      y_val = pd.DataFrame(data = y_val) 

      reshaped_X_train, reshaped_y_train = get_frames(X_train, y_train) 
      reshaped_X_val, reshaped_y_val = get_frames(X_val, y_val) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_2, X_val_2 = X_TRAIN_2[train_ix, :], X_TRAIN_2[val_ix, :]
      y_train_2, y_val_2 = y_TRAIN_2[train_ix], y_TRAIN_2[val_ix] 

      X_train_2 = pd.DataFrame(data = X_train_2, columns = ['x_accelero', 'y_accelero', 'z_accelero','x_gyro', 'y_gyro', 'z_gyro', 'Subject_Trial_Number_Encoded'])
      X_val_2 = pd.DataFrame(data = X_val_2, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      y_train_2 = pd.DataFrame(data = y_train_2)
      y_val_2 = pd.DataFrame(data = y_val_2) 

      reshaped_X_train_2, reshaped_y_train_2 = get_frames(X_train_2, y_train_2) 
      reshaped_X_val_2, reshaped_y_val_2 = get_frames(X_val_2, y_val_2) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_3, X_val_3 = X_TRAIN_3[train_ix, :], X_TRAIN_3[val_ix, :]
      y_train_3, y_val_3 = y_TRAIN_3[train_ix], y_TRAIN_3[val_ix] 

      X_train_3 = pd.DataFrame(data = X_train_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      X_val_3 = pd.DataFrame(data = X_val_3, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      y_train_3 = pd.DataFrame(data = y_train_3)
      y_val_3 = pd.DataFrame(data = y_val_3) 

      reshaped_X_train_3, reshaped_y_train_3 = get_frames(X_train_3, y_train_3) 
      reshaped_X_val_3, reshaped_y_val_3 = get_frames(X_val_3, y_val_3) 

      #-------------------------------------------------------------------------------------------

       # split data
      X_train_4, X_val_4 = X_TRAIN_4[train_ix, :], X_TRAIN_4[val_ix, :]
      y_train_4, y_val_4 = y_TRAIN_4[train_ix], y_TRAIN_4[val_ix] 

      X_train_4 = pd.DataFrame(data = X_train_4, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      X_val_4 = pd.DataFrame(data = X_val_4, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      y_train_4 = pd.DataFrame(data = y_train_4)
      y_val_4 = pd.DataFrame(data = y_val_4) 

      reshaped_X_train_4, reshaped_y_train_4 = get_frames(X_train_4, y_train_4) 
      reshaped_X_val_4, reshaped_y_val_4 = get_frames(X_val_4, y_val_4) 

      #-------------------------------------------------------------------------------------------

      # split data
      X_train_5, X_val_5 = X_TRAIN_5[train_ix, :], X_TRAIN_5[val_ix, :]
      y_train_5, y_val_5 = y_TRAIN_5[train_ix], y_TRAIN_5[val_ix] 

      X_train_5 = pd.DataFrame(data = X_train_5, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      X_val_5 = pd.DataFrame(data = X_val_5, columns = ['x_accelero', 'y_accelero', 'z_accelero', 'x_gyro', 'y_gyro', 'z_gyro','Subject_Trial_Number_Encoded'])
      y_train_5 = pd.DataFrame(data = y_train_5)
      y_val_5 = pd.DataFrame(data = y_val_5) 

      reshaped_X_train_5, reshaped_y_train_5 = get_frames(X_train_5, y_train_5) 
      reshaped_X_val_5, reshaped_y_val_5 = get_frames(X_val_5, y_val_5) 

      #-------------------------------------------------------------------------------------------

      # defining some input variables
      n_timesteps, n_features, n_outputs = reshaped_X_train.shape[1], reshaped_X_val.shape[2], reshaped_y_train.shape[1]

      # getting the model
      model = cnn_model_creation(n_timesteps, n_features)

      # fit model
      history = model.fit([reshaped_X_train,reshaped_X_train_2,reshaped_X_train_3,reshaped_X_train_4,reshaped_X_train_5], reshaped_y_train,
                epochs=50,
                verbose=0,
                callbacks=[lr_callback],
                validation_data=([reshaped_X_val,reshaped_X_val_2,reshaped_X_val_3,reshaped_X_val_4,reshaped_X_val_5], reshaped_y_val)) # Chnage number of epochs!
      
      test_scores = model.evaluate([reshaped_X_test, reshaped_X_test_2,reshaped_X_test_3,reshaped_X_test_4,reshaped_X_test_5],reshaped_y_test, verbose=0)
      acc_per_fold.append(test_scores[1] * 100)
      loss_per_fold.append(test_scores[0])
  mean_accuracy = np.mean(acc_per_fold)
  mean_std = np.std(acc_per_fold)
  mean_loss = np.mean(loss_per_fold)
  print('Sensor:', sensor_position_number, 'Activity:', activity_number)
  print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
  print(f'> Loss: {np.mean(loss_per_fold)}')
  return mean_accuracy

### Call the Train Function

In [ ]:
test_scores = []
activity_list = [1, 2, 3, 4, 5, 6, 7]
sensor_list = [
               [1,2,3,4,5]
               ]
for activity in activity_list:
  for sensor in sensor_list:
    mean_accuracy = get_scores(sensor, activity, final_df)     
    test_scores.append((activity, sensor, mean_accuracy))

test_scores